In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &

!ollama pull mistral

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.



In [3]:
!pip install ollama

In [4]:
!pip install langchain langchain-community ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [13]:
import json, pathlib

#Sample FAQ Dataset
faq_data = [

  {"question": "What is the enrollment procedure?", "answer": "Open the course page, click 'Enroll Now', and pay via bKash, Nagad, Rocket, or card. Access is granted instantly after successful payment."},
  {"question": "How much is the enrollment fee?", "answer": "Courses start from ৳500. Examples: Web Development Basics ৳1,200, Graphic Design Essentials ৳1,000, Digital Marketing Masterclass ৳1,500."},
  {"question": "Do you provide free courses?", "answer": "Yes—'Introduction to Python' and 'Fundamentals of Freelancing' are available free with registration."},
  {"question": "How do I join live classes?", "answer": "You will receive Zoom/Google Meet links via email/SMS and in your dashboard 24 hours before class."},
  {"question": "What happens if I miss a live class?", "answer": "A recording is uploaded to your dashboard within 24 hours and stays available for lifetime access."},
  {"question": "Do I get a certificate?", "answer": "Yes, a digital certificate is issued after completing all lessons and passing the final quiz."},
  {"question": "Is the certificate recognized?", "answer": "Certificates are accepted by many local employers and can be added to your LinkedIn profile."},
  {"question": "Can I switch to another course after enrolling?", "answer": "Yes, within 3 days of enrollment if you’ve completed under 10% of lessons."},
  {"question": "What is the refund policy?", "answer": "Refunds are available within 7 days of purchase if under 20% of the course is completed."},
  {"question": "Are classes in Bangla or English?", "answer": "Most beginner courses are in Bangla; advanced tracks may use English or a Bangla-English mix."},
  {"question": "How do I contact support?", "answer": "Call 09638-123-123 (9 AM–9 PM) or email support@courses.com.bd."},
  {"question": "Can I access courses on mobile or offline?", "answer": "Yes, via our app or mobile browser. Videos can be downloaded in-app for offline viewing."}

  ]

pathlib.Path("data").mkdir(exist_ok=True)
with open("data/faq.json", "w") as f:
    json.dump(faq_data, f, indent=2)

# Load JSON
with open("data/faq.json", "r") as f:
    faq_data = json.load(f)


In [19]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Initialize the Ollama model
chat_model = ChatOllama(model="mistral", temperature=0.7)

# Create a template for chatbot responses
faq_template = PromptTemplate(
    input_variables=["faq_data", "question"],
    template=(
        "You are an AI chatbot. Answer user questions concisely using ONLY the information provided in the FAQ data.\n"
        "Don't mention that your answers come from the FAQ dat.\n"
        "If the user question is related to a topic in the FAQ but the Exact Answer is not Specified, then reply: Please contact us at our customer care.\n"
        "If the question is completely unrelated or not covered in the FAQ, reply: I don’t know, please refine your question.\n"
        "FAQ Data:\n{faq_data}\n\n"
        "Question: {question}"
    )
)

# Create a chain for handling chatbot interactions
faq_chain = LLMChain(
    prompt=faq_template,
    llm=chat_model
)

# Function to handle user queries
def chatbot():
    print("Welcome to the FAQ Chatbot! Ask me about Courses, payments or others enrollment procedure. Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        response = faq_chain.run({"question": user_input, "faq_data": faq_data})
        print("Bot:", response)


In [20]:
chatbot()

Welcome to the FAQ Chatbot! Ask me about Courses, payments or others enrollment procedure. Type 'exit' to stop.
You: hello, wanna know the enrollment procedure
Bot:  To enroll in a course, open the desired course page, click on 'Enroll Now', and pay using bKash, Nagad, Rocket, or card. Access is granted instantly upon successful payment.
You: could you repeat this step by step?
Bot:  To enroll in a course, follow these steps:
1. Open the course page.
2. Click on 'Enroll Now'.
3. Choose your preferred payment method from bKash, Nagad, Rocket, or card.
4. Make the payment.
5. Access to the course will be granted immediately after successful payment.
You: owh. nice. do you accept the apple pay ?
Bot:  No, we currently do not accept Apple Pay for course enrollments. You can pay via bKash, Nagad, Rocket, or card instead.
You: any trial class available for check the Quality?
Bot:  No, there are no trial classes available for checking the quality, but you can enroll in the free courses such a